# AlexNet in Keras


In this notebook, we leverage an [AlexNet](https://papers.nips.cc/paper/4824-imagenet-classification-with-deep-convolutional-neural-networks)-like deep, convolutional neural network to classify flowers into the 17 categories of the [Oxford Flowers](http://www.robots.ox.ac.uk/~vgg/data/flowers/17/) data set. Derived from [this earlier notebook](https://github.com/the-deep-learners/TensorFlow-LiveLessons/blob/master/notebooks/old/L3-3b__TFLearn_AlexNet.ipynb).


[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/NSC-BS-CS/CSB410_Labs/blob/main/BookLabs/7-alexnet_in_keras.ipynb)


#### Load dependencies


In [1]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.layers import BatchNormalization

#### Load _and preprocess_ data


In [8]:
import tensorflow as tf
import tensorflow_datasets as tfds

# Load the Oxford Flowers 102 dataset
# as_supervised=True loads data in (image, label) format
# with_info=True provides dataset metadata
(ds_train, ds_test), ds_info = tfds.load(
    'oxford_flowers102', # Corrected dataset name
    split=['train', 'validation'], # Using validation split as test set for demonstration
    as_supervised=True,
    with_info=True,
    shuffle_files=True
)

# Define preprocessing function
def preprocess_image(image, label):
    # Resize image to the desired size (224x224)
    image = tf.image.resize(image, (224, 224))
    # Normalize image to [0, 1]
    image = tf.cast(image, tf.float32) / 255.0
    # Convert label to one-hot encoding using the correct number of classes
    label = tf.one_hot(label, ds_info.features['label'].num_classes)
    return image, label

# Apply preprocessing to the datasets
ds_train = ds_train.map(preprocess_image).batch(32).prefetch(tf.data.AUTOTUNE)
ds_test = ds_test.map(preprocess_image).batch(32).prefetch(tf.data.AUTOTUNE)

# Note: The original code used X and Y numpy arrays.
# With tf.data.Dataset, you feed the dataset directly to model.fit.

#### Design neural network architecture


In [7]:
model = Sequential()

model.add(Conv2D(96, kernel_size=(11, 11), strides=(4, 4), activation='relu', input_shape=(224, 224, 3)))
model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
model.add(BatchNormalization())

model.add(Conv2D(256, kernel_size=(5, 5), activation='relu'))
model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
model.add(BatchNormalization())

model.add(Conv2D(256, kernel_size=(3, 3), activation='relu'))
model.add(Conv2D(384, kernel_size=(3, 3), activation='relu'))
model.add(Conv2D(384, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
model.add(BatchNormalization())

model.add(Flatten())
model.add(Dense(4096, activation='tanh'))
model.add(Dropout(0.5))
model.add(Dense(4096, activation='tanh'))
model.add(Dropout(0.5))

# Updated to 102 output units for oxford_flowers102 dataset
model.add(Dense(102, activation='softmax'))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [4]:
#model.summary()

#### Configure model


In [9]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

#### Train!


In [10]:
model.fit(ds_train, epochs=100, verbose=1, validation_data=ds_test)

Epoch 1/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 30s 578ms/step - accuracy: 0.0217 - loss: 5.6333 - val_accuracy: 0.0098 - val_loss: 8.6864
Epoch 2/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - accuracy: 0.0416 - loss: 5.8346 - val_accuracy: 0.0157 - val_loss: 6.9989
Epoch 3/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - accuracy: 0.0332 - loss: 5.4832 - val_accuracy: 0.0216 - val_loss: 7.9177
Epoch 4/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - accuracy: 0.0510 - loss: 5.2665 - val_accuracy: 0.0255 - val_loss: 7.4569
Epoch 5/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - accuracy: 0.0664 - loss: 5.1053 - val_accuracy: 0.0167 - val_loss: 7.7825
Epoch 6/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - accuracy: 0.0972 - loss: 4.8651 - val_accuracy: 0.0206 - val_loss: 8.5955
Epoch 7/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - accuracy: 0.0981 - loss: 4.8388 - val_accuracy: 0.0225 - val_loss: 8.4772
Epoch 8/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - accuracy: 0.1005 - loss: 4.6733 - val_accuracy: 